In [1]:
import Orange
Orange.version.version

C:\Users\panov\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'3.13.0'

In [2]:
data = Orange.data.Table("voting")
data

[[n, y, n, y, y, ... | republican],
 [n, y, n, y, y, ... | republican],
 [?, y, y, ?, y, ... | democrat],
 [n, y, y, n, ?, ... | democrat],
 [y, y, y, n, y, ... | democrat],
 ...
]

In [3]:
lr = Orange.classification.LogisticRegressionLearner()
rf = Orange.classification.RandomForestLearner(n_estimators=100)
res = Orange.evaluation.CrossValidation(data, [lr, rf], k=5)

In [4]:
print("Accuracy:", Orange.evaluation.scoring.CA(res))
print("AUC:", Orange.evaluation.scoring.AUC(res))

Accuracy: [0.95862069 0.96091954]
AUC: [0.99237118 0.99038246]


In [5]:
data.domain

[handicapped-infants, water-project-cost-sharing, adoption-of-the-budget-resolution, physician-fee-freeze, el-salvador-aid, religious-groups-in-schools, anti-satellite-test-ban, aid-to-nicaraguan-contras, mx-missile, immigration, synfuels-corporation-cutback, education-spending, superfund-right-to-sue, crime, duty-free-exports, export-administration-act-south-africa | party]

In [6]:
learner = Orange.classification.LogisticRegressionLearner()
classifier = learner(data)

In [7]:
classifier(data[:3])

array([0., 0., 1.])

In [8]:
classifier = Orange.classification.LogisticRegressionLearner(data)
c_values = data.domain.class_var.values
for d in data[5:8]:
    c = classifier(d)
    print("{}, originally {}".format(c_values[int(classifier(d)[0])], d.get_class()))

democrat, originally democrat
republican, originally democrat
republican, originally republican


C:\Users\panov\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: OrangeDeprecationWarning: Passing data into LogisticRegressionLearner constructor is deprecated. Instead, first make an instance, i.e. LogisticRegressionLearner(), then call it with data.
  """Entry point for launching an IPython kernel.


In [9]:
import numpy as np
np.sum(data.Y != classifier(data))

11

In [10]:
len(data)

435

In [11]:
target_class = 1
print("Probabilities for %s:" % data.domain.class_var.values[target_class])
probabilities = classifier(data, 1)
for p, d in zip(probabilities[5:8], data[5:8]):
    print(p[target_class], d.get_class())

Probabilities for democrat:
0.9951828171078181 democrat
0.16702490870039302 democrat
0.08122212559419023 republican


* Разделить выборку на тестовую и обучающую
* Обучить классификаторы: дерево принятия решений, логистическую регрессию и ближайших соседей
* Вывести таблицу вероятностей классов для республиканцев в тестовой выборке 
* Посчитать accuracy и AUC

In [12]:
import random
random.seed(42)
data = Orange.data.Table("voting")
test = Orange.data.Table(data.domain, random.sample(data, 5))
train = Orange.data.Table(data.domain, [d for d in data if d not in test])

tree = Orange.classification.tree.TreeLearner(max_depth=3)
knn = Orange.classification.knn.KNNLearner(n_neighbors=3)
lr = Orange.classification.LogisticRegressionLearner(C=0.1)

learners = [tree, knn, lr]
classifiers = [learner(train) for learner in learners]

target = 0
print("Probabilities for %s:" % data.domain.class_var.values[target])
print("original class ", " ".join("%-5s" % l.name for l in classifiers))

c_values = data.domain.class_var.values
for d in test:
    print(("{:<15}" + " {:.3f}"*len(classifiers)).format(c_values[int(d.get_class())], *(c(d, 1)[0][target] for c in classifiers)))

Probabilities for republican:
original class  tree  knn   logistic regression
republican      0.975 1.000 0.917
republican      0.975 1.000 0.947
democrat        0.008 0.000 0.028
republican      0.975 1.000 0.937
republican      0.975 0.667 0.793


In [13]:
data = Orange.data.Table("housing.tab")
tree_learner = Orange.classification.SimpleTreeLearner(max_depth=2)
tree = tree_learner(data)
print(tree.to_string())


RM (22.5: 506.0)
: <=6.941
   LSTAT (19.9: 430.0)
   : <=14.4 --> (23.3: 255.0)
   : >14.4 --> (15.0: 175.0)
: >6.941
   RM (37.2: 76.0)
   : <=7.437 --> (32.1: 46.0)
   : >7.437 --> (45.1: 30.0)


In [14]:
import random
random.seed(42)
data = Orange.data.Table("housing")

* Разделить выборку на тестовую и обучающую
* Обучить регрессоры: LinearRegressionLearner, RandomForestRegressionLearner, RidgeRegressionLearner
* Проверить на тестовой выборке

In [15]:
# ваш код
data = Orange.data.Table("housing")
test = Orange.data.Table(data.domain, random.sample(data, 5))
train = Orange.data.Table(data.domain, [d for d in data if d not in test])

lin = Orange.regression.linear.LinearRegressionLearner()
rf = Orange.regression.random_forest.RandomForestRegressionLearner()
rf.name = "rf"
ridge = Orange.regression.RidgeRegressionLearner()

learners = [lin, rf, ridge]
regressors = [learner(train) for learner in learners]

print("y   ", " ".join("%5s" % l.name for l in regressors))

for d in test:
    print(("{:<5}" + " {:5.1f}"*len(regressors)).format(d.get_class(), *(r(d)[0] for r in regressors)))

y    linear regression    rf ridge regression
22.2   19.3  20.3  19.5
31.6   33.2  31.7  33.2
21.7   20.9  20.9  21.0
10.2   16.9  12.0  16.8
14.0   13.6  14.8  13.5
